<a href="https://colab.research.google.com/github/nmansour67/skills-introduction-to-github/blob/main/Oman_UAE_Trade_Analysis_ReportNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ============================================================================
# CELL 1: SETUP & INSTALLATION
# ============================================================================
print("="*80)
print("COMPREHENSIVE BILATERAL TRADE ANALYSIS - ALL DATA SOURCES")
print("="*80)

import sys
import subprocess

packages = ['python-docx', 'openpyxl', 'scikit-learn', 'scipy', 'pandas', 'numpy']
print("\n📦 Installing packages...")
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup complete!")

COMPREHENSIVE BILATERAL TRADE ANALYSIS - ALL DATA SOURCES

📦 Installing packages...
✅ Setup complete!


In [6]:
# ============================================================================
# CELL 2: UPLOAD ALL DATA FILES
# ============================================================================
from google.colab import files
import os

print("="*80)
print("📁 UPLOAD ALL YOUR DATA FILES")
print("="*80)
print("\nPlease upload these 4 files:")
print("  1. Trade_Balance_Oman_UAE.csv")
print("  2. Oman_Export_to_UAE_HS6_10Yrs_Value.numbers")
print("  3. Oman_Import_from_UAE_HS6_10Yrs_Value.numbers")
print("  4. Oman_Re-export_to_UAE_HS6_10Yrs_Value.numbers")
print("\n(Select all 4 files at once)\n")

uploaded = files.upload()

print("\n" + "="*80)
print("📦 FILES UPLOADED:")
print("="*80)
for filename, content in uploaded.items():
    print(f"  ✓ {filename} ({len(content):,} bytes)")

# Identify files
balance_file = None
export_file = None
import_file = None
reexport_file = None

for filename in uploaded.keys():
    if 'balance' in filename.lower():
        balance_file = filename
    elif 'export' in filename.lower() and 're-export' not in filename.lower():
        export_file = filename
    elif 'import' in filename.lower():
        import_file = filename
    elif 're-export' in filename.lower():
        reexport_file = filename

print("\n📋 FILE MAPPING:")
print(f"  Balance:    {balance_file}")
print(f"  Exports:    {export_file}")
print(f"  Imports:    {import_file}")
print(f"  Re-exports: {reexport_file}")
print("="*80)

📁 UPLOAD ALL YOUR DATA FILES

Please upload these 4 files:
  1. Trade_Balance_Oman_UAE.csv
  2. Oman_Export_to_UAE_HS6_10Yrs_Value.numbers
  3. Oman_Import_from_UAE_HS6_10Yrs_Value.numbers
  4. Oman_Re-export_to_UAE_HS6_10Yrs_Value.numbers

(Select all 4 files at once)



Saving Trade Balance Oman UAE.csv to Trade Balance Oman UAE (1).csv
Saving Oman Re-export to UAE HS6 10Yrs Value.numbers to Oman Re-export to UAE HS6 10Yrs Value (1).numbers
Saving Oman Import from UAE HS6 10Yrs Value.numbers to Oman Import from UAE HS6 10Yrs Value (1).numbers
Saving Oman Export to UAE HS6 10Yrs Value.numbers to Oman Export to UAE HS6 10Yrs Value (1).numbers

📦 FILES UPLOADED:
  ✓ Trade Balance Oman UAE (1).csv (1,031 bytes)
  ✓ Oman Re-export to UAE HS6 10Yrs Value (1).numbers (201,942 bytes)
  ✓ Oman Import from UAE HS6 10Yrs Value (1).numbers (194,100 bytes)
  ✓ Oman Export to UAE HS6 10Yrs Value (1).numbers (199,500 bytes)

📋 FILE MAPPING:
  Balance:    Trade Balance Oman UAE (1).csv
  Exports:    Oman Export to UAE HS6 10Yrs Value (1).numbers
  Imports:    Oman Import from UAE HS6 10Yrs Value (1).numbers
  Re-exports: Oman Re-export to UAE HS6 10Yrs Value (1).numbers


In [7]:
# ============================================================================
# CELL 3: CONVERT .NUMBERS FILES TO CSV
# ============================================================================

print("\n🔄 Converting .numbers files to CSV format...")
print("="*80)

# Install numbers-parser
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numbers-parser"])

from numbers_parser import Document as NumbersDocument

def convert_numbers_to_dataframe(filename):
    """Convert .numbers file to pandas DataFrame"""
    try:
        doc = NumbersDocument(filename)
        sheets = doc.sheets

        if len(sheets) == 0:
            print(f"  ⚠️  No sheets found in {filename}")
            return None

        # Get first sheet and first table
        sheet = sheets[0]
        tables = sheet.tables

        if len(tables) == 0:
            print(f"  ⚠️  No tables found in {filename}")
            return None

        table = tables[0]

        # Extract data
        data = []
        num_rows = table.num_rows
        num_cols = table.num_cols

        for row in range(num_rows):
            row_data = []
            for col in range(num_cols):
                cell = table.cell(row, col)
                row_data.append(cell.value if cell.value is not None else '')
            data.append(row_data)

        # Create DataFrame
        df = pd.DataFrame(data)

        # Try to set first row as header
        if len(df) > 0:
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)

        return df

    except Exception as e:
        print(f"  ❌ Error converting {filename}: {str(e)}")
        return None

# Convert export file
if export_file:
    print(f"\n📊 Processing: {export_file}")
    df_exports = convert_numbers_to_dataframe(export_file)
    if df_exports is not None:
        print(f"  ✓ Shape: {df_exports.shape}")
        print(f"  ✓ Columns: {list(df_exports.columns[:5])}...")
        df_exports.to_csv('/content/exports_converted.csv', index=False)
    else:
        df_exports = pd.DataFrame()

# Convert import file
if import_file:
    print(f"\n📊 Processing: {import_file}")
    df_imports = convert_numbers_to_dataframe(import_file)
    if df_imports is not None:
        print(f"  ✓ Shape: {df_imports.shape}")
        print(f"  ✓ Columns: {list(df_imports.columns[:5])}...")
        df_imports.to_csv('/content/imports_converted.csv', index=False)
    else:
        df_imports = pd.DataFrame()

# Convert re-export file
if reexport_file:
    print(f"\n📊 Processing: {reexport_file}")
    df_reexports = convert_numbers_to_dataframe(reexport_file)
    if df_reexports is not None:
        print(f"  ✓ Shape: {df_reexports.shape}")
        print(f"  ✓ Columns: {list(df_reexports.columns[:5])}...")
        df_reexports.to_csv('/content/reexports_converted.csv', index=False)
    else:
        df_reexports = pd.DataFrame()

print("\n✅ Conversion complete!")
print("="*80)


🔄 Converting .numbers files to CSV format...

📊 Processing: Oman Export to UAE HS6 10Yrs Value (1).numbers
  ✓ Shape: (21, 12)
  ✓ Columns: ['Product code', 'Product label', np.float64(2015.0), np.float64(2016.0), np.float64(2017.0)]...

📊 Processing: Oman Import from UAE HS6 10Yrs Value (1).numbers
  ✓ Shape: (21, 12)
  ✓ Columns: ['Product code', 'Product label', np.float64(2015.0), np.float64(2016.0), np.float64(2017.0)]...

📊 Processing: Oman Re-export to UAE HS6 10Yrs Value (1).numbers
  ✓ Shape: (21, 11)
  ✓ Columns: ['Product code', 'Product label', np.float64(2015.0), np.float64(2016.0), np.float64(2017.0)]...

✅ Conversion complete!


In [8]:
# ============================================================================
# CELL 4: LOAD TRADE BALANCE DATA
# ============================================================================

def clean_value(val):
    """Clean currency values"""
    if pd.isna(val) or val == '':
        return 0.0
    if isinstance(val, (int, float)):
        return float(val)
    val_str = str(val).replace('(', '-').replace(')', '').replace('$', '').replace(',', '').strip()
    try:
        return float(val_str)
    except:
        return 0.0

print("\n📊 Loading trade balance data...")
df = pd.read_csv(balance_file)

# Extract years
year_cols = [col for col in df.columns if col.startswith('20') and len(col) == 4]
print(f"  Years found: {', '.join(year_cols)}")

# Extract rows
exports_row = df[df.iloc[:, 0].str.contains('Export.*to', case=False, na=False)]
reexports_row = df[df.iloc[:, 0].str.contains('Re-export.*to', case=False, na=False)]
imports_row = df[df.iloc[:, 0].str.contains('Import.*from', case=False, na=False)]

# Build dataframe
trade_data = {}
for year in year_cols:
    trade_data[year] = {
        'Exports': clean_value(exports_row[year].iloc[0] if len(exports_row) > 0 else 0),
        'Re-exports': clean_value(reexports_row[year].iloc[0] if len(reexports_row) > 0 else 0),
        'Imports': clean_value(imports_row[year].iloc[0] if len(imports_row) > 0 else 0)
    }

df_trade = pd.DataFrame(trade_data).T
df_trade['Total_Exports'] = df_trade['Exports'] + df_trade['Re-exports']
df_trade['Trade_Balance'] = df_trade['Total_Exports'] - df_trade['Imports']
df_trade['Coverage_Ratio'] = (df_trade['Total_Exports'] / df_trade['Imports'] * 100)
df_trade['Trade_Volume'] = df_trade['Total_Exports'] + df_trade['Imports']

print("\n✅ Trade balance data loaded!")
print("\n📋 TRADE BALANCE SUMMARY:")
print(df_trade)


📊 Loading trade balance data...
  Years found: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023

✅ Trade balance data loaded!

📋 TRADE BALANCE SUMMARY:
           Exports    Re-exports       Imports  Total_Exports  Trade_Balance  \
2015  2.050237e+09  2.535346e+09  1.015989e+10   4.585583e+09  -5.574303e+09   
2016  2.102307e+09  5.298140e+08  3.457232e+09   2.632121e+09  -8.251110e+08   
2017  2.255425e+09  4.060300e+08  1.028272e+10   2.661455e+09  -7.621265e+09   
2018  2.901981e+09  1.114220e+09  1.182290e+10   4.016201e+09  -7.806704e+09   
2019  2.698993e+09  1.118703e+09  9.471186e+09   3.817696e+09  -5.653490e+09   
2020  3.680196e+09  1.541694e+09  1.048976e+10   5.221890e+09  -5.267866e+09   
2021  3.843455e+09  1.277884e+09  1.102288e+10   5.121339e+09  -5.901536e+09   
2022  3.143137e+09  1.135879e+09  1.117215e+10   4.279016e+09  -6.893130e+09   
2023  3.760126e+09  1.309436e+09  1.020805e+10   5.069562e+09  -5.138485e+09   

      Coverage_Ratio  Trade_Volume  
2015 

In [9]:
# ============================================================================
# CELL 5: PROCESS PRODUCT-LEVEL DATA (HS6)
# ============================================================================

print("\n🔍 Processing product-level data...")
print("="*80)

# Check if we have product data
has_product_data = (not df_exports.empty or not df_imports.empty or not df_reexports.empty)

if has_product_data:
    print("✅ Product-level data available for analysis")

    # Display sample from exports
    if not df_exports.empty:
        print(f"\n📦 EXPORT PRODUCTS:")
        print(f"   Total products: {len(df_exports)}")
        print(f"   Sample data:")
        print(df_exports.head(3))

    # Display sample from imports
    if not df_imports.empty:
        print(f"\n📦 IMPORT PRODUCTS:")
        print(f"   Total products: {len(df_imports)}")
        print(f"   Sample data:")
        print(df_imports.head(3))

    # Display sample from re-exports
    if not df_reexports.empty:
        print(f"\n📦 RE-EXPORT PRODUCTS:")
        print(f"   Total products: {len(df_reexports)}")
        print(f"   Sample data:")
        print(df_reexports.head(3))

    # Find year columns in product data
    if not df_exports.empty:
        product_year_cols = [col for col in df_exports.columns if str(col).startswith('20') or str(col).isdigit()]
        print(f"\n   Year columns found: {product_year_cols[:5]}")

else:
    print("⚠️  Product-level data not available - using aggregate analysis only")
    product_year_cols = []

print("="*80)


🔍 Processing product-level data...
✅ Product-level data available for analysis

📦 EXPORT PRODUCTS:
   Total products: 21
   Sample data:
0 Product code                                      Product label  \
0       'TOTAL                                       All products   
1      '721420  Bars and rods, of iron or non-alloy steel, wit...   
2      '271012  Light oils and preparations, of petroleum or b...   

0        2015.0        2016.0        2017.0        2018.0        2019.0  \
0  2.050237e+09  2.102307e+09  2.255425e+09  2.901981e+09  2.698993e+09   
1  6.864000e+06  2.546100e+07  2.736800e+08  5.440940e+08  3.344430e+08   
2  3.031330e+08  1.443700e+08  2.005330e+08  1.946800e+07  9.431900e+07   

0        2020.0        2021.0        2022.0        2023.0    
0  3.680196e+09  3.843455e+09  3.143137e+09  3.760126e+09    
1  2.439330e+08  2.326460e+08  1.852170e+08  3.435450e+08    
2  4.204500e+07  6.032000e+06  2.633700e+07  2.850310e+08    

📦 IMPORT PRODUCTS:
   Total product

In [10]:
# ============================================================================
# CELL 6: BCG MATRIX ANALYSIS
# ============================================================================

print("\n📊 BCG MATRIX ANALYSIS")
print("="*80)

def calculate_product_cagr(df_products, product_col='Product', year_cols=None):
    """Calculate CAGR for each product"""
    if year_cols is None or len(year_cols) < 2:
        return pd.DataFrame()

    results = []

    for idx, row in df_products.iterrows():
        try:
            # Get first and last year values
            first_year = year_cols[0]
            last_year = year_cols[-1]

            start_val = pd.to_numeric(row[first_year], errors='coerce')
            end_val = pd.to_numeric(row[last_year], errors='coerce')

            if pd.notna(start_val) and pd.notna(end_val) and start_val > 0 and end_val > 0:
                periods = len(year_cols) - 1
                cagr = (((end_val / start_val) ** (1 / periods)) - 1) * 100

                # Get total value
                total_val = sum([pd.to_numeric(row[y], errors='coerce') for y in year_cols if pd.notna(pd.to_numeric(row[y], errors='coerce'))])

                results.append({
                    'Product': row.get(product_col, row.iloc[0]),
                    'Start_Value': start_val,
                    'End_Value': end_val,
                    'CAGR': cagr,
                    'Total_Value': total_val
                })
        except:
            continue

    return pd.DataFrame(results)

# Calculate for exports if available
bcg_export = pd.DataFrame()
bcg_import = pd.DataFrame()

if has_product_data and len(product_year_cols) >= 2:
    print("Calculating product growth rates...")

    if not df_exports.empty:
        bcg_export = calculate_product_cagr(df_exports, year_cols=product_year_cols)
        print(f"  ✓ Processed {len(bcg_export)} export products")

    if not df_imports.empty:
        bcg_import = calculate_product_cagr(df_imports, year_cols=product_year_cols)
        print(f"  ✓ Processed {len(bcg_import)} import products")

    # BCG Classification for exports
    if not bcg_export.empty:
        # Calculate market share (as % of total)
        total_export_value = bcg_export['Total_Value'].sum()
        bcg_export['Market_Share'] = (bcg_export['Total_Value'] / total_export_value) * 100

        # Get medians
        median_growth = bcg_export['CAGR'].median()
        median_share = bcg_export['Market_Share'].median()

        # Classify
        def classify_bcg(row):
            if row['CAGR'] >= median_growth and row['Market_Share'] >= median_share:
                return 'Star'
            elif row['CAGR'] < median_growth and row['Market_Share'] >= median_share:
                return 'Cash Cow'
            elif row['CAGR'] >= median_growth and row['Market_Share'] < median_share:
                return 'Question Mark'
            else:
                return 'Dog'

        bcg_export['BCG_Category'] = bcg_export.apply(classify_bcg, axis=1)

        # Summary
        bcg_summary = bcg_export.groupby('BCG_Category').agg({
            'Product': 'count',
            'Total_Value': 'sum',
            'CAGR': 'mean'
        }).round(2)

        print("\n📊 BCG MATRIX SUMMARY (Exports):")
        print(bcg_summary)

        print("\n🌟 TOP STARS (High Growth, High Share):")
        stars = bcg_export[bcg_export['BCG_Category'] == 'Star'].nlargest(5, 'Total_Value')
        print(stars[['Product', 'CAGR', 'Market_Share', 'Total_Value']])

else:
    print("⚠️  Insufficient product data for BCG analysis")
    bcg_summary = pd.DataFrame()

print("="*80)


📊 BCG MATRIX ANALYSIS
Calculating product growth rates...
  ✓ Processed 12 export products
  ✓ Processed 18 import products

📊 BCG MATRIX SUMMARY (Exports):
               Product   Total_Value   CAGR
BCG_Category                               
Cash Cow             4  2.927780e+10   1.79
Dog                  2  5.424770e+08   1.62
Question Mark        4  8.035490e+08  93.77
Star                 2  2.731384e+09  52.82

🌟 TOP STARS (High Growth, High Share):
   Product       CAGR  Market_Share   Total_Value
1  '721420  63.089422      6.565341  2.189883e+09
7  '710812  42.544155      1.623438  5.415010e+08


In [11]:
# ============================================================================
# CELL 7: HHI CONCENTRATION ANALYSIS
# ============================================================================

print("\n📊 HHI CONCENTRATION INDEX")
print("="*80)

def calculate_hhi(df_products, value_col='Total_Value'):
    """Calculate Herfindahl-Hirschman Index"""
    if df_products.empty or value_col not in df_products.columns:
        return 0

    total = df_products[value_col].sum()
    if total == 0:
        return 0

    # Calculate market shares (as percentages)
    shares = (df_products[value_col] / total) * 100

    # HHI = sum of squared shares
    hhi = (shares ** 2).sum()

    return hhi

if has_product_data:
    # Calculate HHI for exports
    if not bcg_export.empty:
        hhi_exports = calculate_hhi(bcg_export, 'Total_Value')

        # Interpret HHI
        if hhi_exports < 1500:
            concentration = "Low (Competitive market)"
        elif hhi_exports < 2500:
            concentration = "Moderate concentration"
        else:
            concentration = "High concentration"

        print(f"EXPORT CONCENTRATION:")
        print(f"  HHI Index: {hhi_exports:.0f}")
        print(f"  Assessment: {concentration}")

        # Top products by market share
        top_products = bcg_export.nlargest(10, 'Market_Share')
        top_share_sum = top_products['Market_Share'].sum()

        print(f"\n  Top 10 products control {top_share_sum:.1f}% of export value")
        print(f"\n  TOP 5 EXPORT PRODUCTS:")
        for idx, row in top_products.head(5).iterrows():
            print(f"    • {row['Product'][:50]}: {row['Market_Share']:.1f}% (${row['Total_Value']/1e6:.1f}M)")

    # Calculate HHI for imports
    if not bcg_import.empty:
        hhi_imports = calculate_hhi(bcg_import, 'Total_Value')

        if hhi_imports < 1500:
            concentration_imp = "Low (Competitive)"
        elif hhi_imports < 2500:
            concentration_imp = "Moderate"
        else:
            concentration_imp = "High"

        print(f"\n\nIMPORT CONCENTRATION:")
        print(f"  HHI Index: {hhi_imports:.0f}")
        print(f"  Assessment: {concentration_imp}")

        # Add market share to imports
        total_import_value = bcg_import['Total_Value'].sum()
        bcg_import['Market_Share'] = (bcg_import['Total_Value'] / total_import_value) * 100

        top_imports = bcg_import.nlargest(10, 'Market_Share')
        top_import_share = top_imports['Market_Share'].sum()

        print(f"\n  Top 10 products control {top_import_share:.1f}% of import value")
        print(f"\n  TOP 5 IMPORT PRODUCTS:")
        for idx, row in top_imports.head(5).iterrows():
            print(f"    • {row['Product'][:50]}: {row['Market_Share']:.1f}% (${row['Total_Value']/1e6:.1f}M)")
else:
    print("⚠️  Product data not available for HHI calculation")
    hhi_exports = 0
    hhi_imports = 0

print("="*80)


📊 HHI CONCENTRATION INDEX
EXPORT CONCENTRATION:
  HHI Index: 6355
  Assessment: High concentration

  Top 10 products control 98.9% of export value

  TOP 5 EXPORT PRODUCTS:
    • 'TOTAL: 79.3% ($26435.9M)
    • '721420: 6.6% ($2189.9M)
    • '271012: 3.4% ($1121.3M)
    • '854449: 2.8% ($943.1M)
    • '720711: 2.3% ($777.5M)


IMPORT CONCENTRATION:
  HHI Index: 6507
  Assessment: High

  Top 10 products control 96.1% of import value

  TOP 5 IMPORT PRODUCTS:
    • 'TOTAL: 80.4% ($88086.8M)
    • '710812: 3.2% ($3543.6M)
    • '271012: 2.6% ($2839.5M)
    • '740811: 2.5% ($2733.2M)
    • '711319: 1.8% ($1938.9M)


In [12]:
# ============================================================================
# CELL 8: CALCULATE AGGREGATE METRICS
# ============================================================================

print("\n📊 CALCULATING COMPREHENSIVE METRICS")
print("="*80)

# CAGR function
def calc_cagr(start, end, periods):
    if start == 0 or end == 0:
        return 0.0
    return (((end / start) ** (1 / periods)) - 1) * 100

years = len(df_trade)

# Growth rates
cagr_exports = calc_cagr(df_trade['Exports'].iloc[0], df_trade['Exports'].iloc[-1], years-1)
cagr_reexports = calc_cagr(df_trade['Re-exports'].iloc[0], df_trade['Re-exports'].iloc[-1], years-1)
cagr_imports = calc_cagr(df_trade['Imports'].iloc[0], df_trade['Imports'].iloc[-1], years-1)
cagr_volume = calc_cagr(df_trade['Trade_Volume'].iloc[0], df_trade['Trade_Volume'].iloc[-1], years-1)

# Cumulative metrics
cum_exports = df_trade['Total_Exports'].sum()
cum_imports = df_trade['Imports'].sum()
cum_balance = df_trade['Trade_Balance'].sum()
cum_volume = df_trade['Trade_Volume'].sum()

# Balance profile
deficit_years = (df_trade['Trade_Balance'] < 0).sum()
surplus_years = (df_trade['Trade_Balance'] >= 0).sum()
peak_deficit = df_trade['Trade_Balance'].min()
peak_surplus = df_trade['Trade_Balance'].max()

# Coverage stats
avg_coverage = df_trade['Coverage_Ratio'].mean()
latest_coverage = df_trade['Coverage_Ratio'].iloc[-1]

# YoY volatility
yoy_exports = df_trade['Exports'].pct_change() * 100
yoy_imports = df_trade['Imports'].pct_change() * 100
vol_exports = yoy_exports.std()
vol_imports = yoy_imports.std()

# Re-export share
reexport_share = (df_trade['Re-exports'].sum() / df_trade['Total_Exports'].sum()) * 100

# Linear regression for trends
years_array = np.arange(len(df_trade)).reshape(-1, 1)
export_model = LinearRegression().fit(years_array, df_trade['Exports'].values)
import_model = LinearRegression().fit(years_array, df_trade['Imports'].values)

export_r2 = export_model.score(years_array, df_trade['Exports'].values)
import_r2 = import_model.score(years_array, df_trade['Imports'].values)

print("✅ All metrics calculated!")

# Display comprehensive summary
print("\n" + "="*80)
print("COMPREHENSIVE METRICS SUMMARY")
print("="*80)
print(f"\n{'CUMULATIVE (2015-2023):'}")
print(f"  Trade Volume:        ${cum_volume/1e9:>8.1f}B")
print(f"  Total Exports:       ${cum_exports/1e9:>8.1f}B")
print(f"  Total Imports:       ${cum_imports/1e9:>8.1f}B")
print(f"  Trade Balance:       ${cum_balance/1e9:>8.1f}B")

print(f"\n{'GROWTH RATES (CAGR):'}")
print(f"  Exports:             {cagr_exports:>7.1f}%")
print(f"  Re-exports:          {cagr_reexports:>7.1f}%")
print(f"  Imports:             {cagr_imports:>7.1f}%")
print(f"  Trade Volume:        {cagr_volume:>7.1f}%")

print(f"\n{'TRADE BALANCE PROFILE:'}")
print(f"  Deficit Years:       {deficit_years:>7} / {years}")
print(f"  Surplus Years:       {surplus_years:>7} / {years}")
print(f"  Peak Deficit:        ${abs(peak_deficit)/1e9:>7.1f}B")
print(f"  Peak Surplus:        ${peak_surplus/1e9:>7.1f}B")

print(f"\n{'COVERAGE & DEPENDENCY:'}")
print(f"  Latest Coverage:     {latest_coverage:>7.1f}%")
print(f"  Average Coverage:    {avg_coverage:>7.1f}%")
print(f"  Re-export Share:     {reexport_share:>7.1f}%")

print(f"\n{'VOLATILITY & TRENDS:'}")
print(f"  Export Volatility:   {vol_exports:>7.1f}%")
print(f"  Import Volatility:   {vol_imports:>7.1f}%")
print(f"  Export Trend R²:     {export_r2:>7.3f}")
print(f"  Import Trend R²:     {import_r2:>7.3f}")

if has_product_data and not bcg_export.empty:
    print(f"\n{'PRODUCT CONCENTRATION:'}")
    print(f"  Export HHI:          {hhi_exports:>7.0f}")
    print(f"  Import HHI:          {hhi_imports:>7.0f}")

    print(f"\n{'BCG PORTFOLIO (Exports):'}")
    if not bcg_summary.empty:
        for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
            if category in bcg_summary.index:
                count = bcg_summary.loc[category, 'Product']
                value = bcg_summary.loc[category, 'Total_Value']
                print(f"  {category + ':':<18} {count:>3.0f} products (${value/1e9:>6.2f}B)")

print("="*80)


📊 CALCULATING COMPREHENSIVE METRICS
✅ All metrics calculated!

COMPREHENSIVE METRICS SUMMARY

CUMULATIVE (2015-2023):
  Trade Volume:        $   125.5B
  Total Exports:       $    37.4B
  Total Imports:       $    88.1B
  Trade Balance:       $   -50.7B

GROWTH RATES (CAGR):
  Exports:                 7.9%
  Re-exports:             -7.9%
  Imports:                 0.1%
  Trade Volume:            0.4%

TRADE BALANCE PROFILE:
  Deficit Years:             9 / 9
  Surplus Years:             0 / 9
  Peak Deficit:        $    7.8B
  Peak Surplus:        $   -0.8B

COVERAGE & DEPENDENCY:
  Latest Coverage:        49.7%
  Average Coverage:       45.1%
  Re-export Share:        29.3%

VOLATILITY & TRENDS:
  Export Volatility:      18.2%
  Import Volatility:      77.4%
  Export Trend R²:       0.787
  Import Trend R²:       0.188

PRODUCT CONCENTRATION:
  Export HHI:             6355
  Import HHI:             6507

BCG PORTFOLIO (Exports):
  Star:                2 products ($  2.73B)
  Cash Cow

In [13]:
# ============================================================================
# CELL 9: GENERATE ALL VISUALIZATIONS (HIGH QUALITY)
# ============================================================================

print("\n📊 GENERATING COMPREHENSIVE VISUALIZATIONS")
print("="*80)

years_list = [int(y) for y in df_trade.index]

# ========================================================================
# FIGURE 1: TRADE FLOW TRENDS
# ========================================================================
fig1, ax1 = plt.subplots(figsize=(14, 8))
ax1.plot(years_list, df_trade['Exports']/1e9, marker='o', linewidth=3, markersize=10,
         label='Direct Exports', color='#2E86AB')
ax1.plot(years_list, df_trade['Re-exports']/1e9, marker='s', linewidth=3, markersize=10,
         label='Re-exports', color='#A23B72')
ax1.plot(years_list, df_trade['Imports']/1e9, marker='^', linewidth=3, markersize=10,
         label='Imports', color='#F18F01')
ax1.plot(years_list, df_trade['Total_Exports']/1e9, marker='D', linewidth=2.5,
         linestyle='--', label='Total Exports', color='#06A77D')

# Add trend lines
X = np.arange(len(years_list)).reshape(-1, 1)
for col, color, label in [('Exports', '#2E86AB', 'Export Trend'),
                           ('Imports', '#F18F01', 'Import Trend')]:
    y = df_trade[col].values / 1e9
    model = LinearRegression().fit(X, y)
    trend = model.predict(X)
    ax1.plot(years_list, trend, linestyle=':', linewidth=2, color=color, alpha=0.5)

ax1.set_xlabel('Year', fontsize=13, fontweight='bold')
ax1.set_ylabel('Trade Value (USD Billions)', fontsize=13, fontweight='bold')
ax1.set_title('Oman-UAE Trade Flow Trends (2015-2023)', fontsize=16, fontweight='bold', pad=20)
ax1.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(years_list)
plt.tight_layout()
fig1.savefig('/tmp/fig1_trade_flows.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 1: Trade Flow Trends")

# ========================================================================
# FIGURE 2: TRADE BALANCE EVOLUTION
# ========================================================================
fig2, ax2 = plt.subplots(figsize=(14, 8))
balance_values = df_trade['Trade_Balance']/1e9
colors = ['#06A77D' if x >= 0 else '#C73E1D' for x in balance_values]

bars = ax2.bar(years_list, balance_values, color=colors, alpha=0.7,
               edgecolor='black', linewidth=1.5, width=0.6)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=2)

# Add value labels
for year, val in zip(years_list, balance_values):
    ax2.text(year, val + (0.3 if val >= 0 else -0.3), f'${val:.1f}B',
            ha='center', va='bottom' if val >= 0 else 'top',
            fontsize=10, fontweight='bold')

ax2.set_xlabel('Year', fontsize=13, fontweight='bold')
ax2.set_ylabel('Trade Balance (USD Billions)', fontsize=13, fontweight='bold')
ax2.set_title('Oman-UAE Trade Balance Evolution (Including Re-exports)',
             fontsize=16, fontweight='bold', pad=20)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_xticks(years_list)

# Add annotation
avg_balance = balance_values.mean()
ax2.text(0.98, 0.02, f'Average Balance: ${avg_balance:.1f}B',
        transform=ax2.transAxes, ha='right', va='bottom',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
        fontsize=10, fontweight='bold')

plt.tight_layout()
fig2.savefig('/tmp/fig2_balance.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 2: Trade Balance Evolution")

# ========================================================================
# FIGURE 3: COVERAGE RATIO TREND
# ========================================================================
fig3, ax3 = plt.subplots(figsize=(14, 8))
coverage = df_trade['Coverage_Ratio']

ax3.plot(years_list, coverage, marker='o', linewidth=3, color='#2E86AB', markersize=10)
ax3.axhline(y=100, color='red', linestyle='--', linewidth=2.5, label='Equilibrium (100%)')
ax3.fill_between(years_list, coverage, 100, where=(coverage < 100),
                 alpha=0.25, color='#C73E1D', label='Deficit Zone')
ax3.fill_between(years_list, coverage, 100, where=(coverage >= 100),
                 alpha=0.25, color='#06A77D', label='Surplus Zone')

# Add value labels
for year, val in zip(years_list, coverage):
    ax3.text(year, val + 3, f'{val:.1f}%', ha='center', va='bottom',
            fontsize=10, fontweight='bold')

ax3.set_xlabel('Year', fontsize=13, fontweight='bold')
ax3.set_ylabel('Coverage Ratio (%)', fontsize=13, fontweight='bold')
ax3.set_title('Oman Export Coverage of Imports from UAE',
             fontsize=16, fontweight='bold', pad=20)
ax3.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax3.grid(True, alpha=0.3)
ax3.set_xticks(years_list)
ax3.set_ylim([min(coverage)-10, max(coverage)+15])

plt.tight_layout()
fig3.savefig('/tmp/fig3_coverage.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 3: Coverage Ratio Trend")

# ========================================================================
# FIGURE 4: CAGR COMPARISON
# ========================================================================
fig4, ax4 = plt.subplots(figsize=(14, 8))
categories = ['Direct Exports', 'Re-exports', 'Imports', 'Total Exports', 'Trade Volume']
cagrs = [cagr_exports, cagr_reexports, cagr_imports,
         calc_cagr(df_trade['Total_Exports'].iloc[0], df_trade['Total_Exports'].iloc[-1], years-1),
         cagr_volume]
colors_bar = ['#2E86AB', '#A23B72', '#F18F01', '#06A77D', '#DD1C1A']

bars = ax4.barh(categories, cagrs, color=colors_bar, alpha=0.8,
                edgecolor='black', linewidth=1.5, height=0.6)
ax4.axvline(x=0, color='black', linestyle='-', linewidth=2)

# Add value labels
for i, (cat, val) in enumerate(zip(categories, cagrs)):
    ax4.text(val + (0.4 if val >= 0 else -0.4), i, f'{val:.1f}%',
            va='center', ha='left' if val >= 0 else 'right',
            fontsize=11, fontweight='bold')

ax4.set_xlabel('Compound Annual Growth Rate (%)', fontsize=13, fontweight='bold')
ax4.set_title('CAGR Analysis (2015-2023)', fontsize=16, fontweight='bold', pad=20)
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
fig4.savefig('/tmp/fig4_cagr.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 4: CAGR Comparison")

# ========================================================================
# FIGURE 5: EXPORT COMPOSITION (STACKED AREA)
# ========================================================================
fig5, ax5 = plt.subplots(figsize=(14, 8))

ax5.fill_between(years_list, 0, df_trade['Exports']/1e9,
                 alpha=0.7, color='#2E86AB', label='Direct Exports')
ax5.fill_between(years_list, df_trade['Exports']/1e9, df_trade['Total_Exports']/1e9,
                 alpha=0.7, color='#A23B72', label='Re-exports')

# Add percentage labels
for year in years_list:
    total = df_trade.loc[str(year), 'Total_Exports']
    reexp = df_trade.loc[str(year), 'Re-exports']
    pct = (reexp / total * 100) if total > 0 else 0
    y_pos = df_trade.loc[str(year), 'Total_Exports']/1e9 + 0.5
    ax5.text(year, y_pos, f'{pct:.1f}%\nRe-exp', ha='center',
            fontsize=8, fontweight='bold')

ax5.set_xlabel('Year', fontsize=13, fontweight='bold')
ax5.set_ylabel('Export Value (USD Billions)', fontsize=13, fontweight='bold')
ax5.set_title('Oman Export Composition to UAE', fontsize=16, fontweight='bold', pad=20)
ax5.legend(loc='upper left', frameon=True, shadow=True, fontsize=11)
ax5.grid(True, alpha=0.3, axis='y')
ax5.set_xticks(years_list)

plt.tight_layout()
fig5.savefig('/tmp/fig5_composition.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 5: Export Composition")

# ========================================================================
# FIGURE 6: BCG MATRIX (IF AVAILABLE)
# ========================================================================
if has_product_data and not bcg_export.empty:
    fig6, ax6 = plt.subplots(figsize=(14, 10))

    # Plot each category with different colors
    colors_bcg = {
        'Star': '#FFD700',
        'Cash Cow': '#90EE90',
        'Question Mark': '#FF6B6B',
        'Dog': '#D3D3D3'
    }

    for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
        data = bcg_export[bcg_export['BCG_Category'] == category]
        if not data.empty:
            ax6.scatter(data['Market_Share'], data['CAGR'],
                       s=data['Total_Value']/1e6,  # Size by value
                       c=colors_bcg[category], alpha=0.6, edgecolors='black',
                       linewidth=1.5, label=category)

    # Add median lines
    ax6.axvline(x=bcg_export['Market_Share'].median(), color='black',
               linestyle='--', linewidth=2, alpha=0.5)
    ax6.axhline(y=bcg_export['CAGR'].median(), color='black',
               linestyle='--', linewidth=2, alpha=0.5)

    # Add quadrant labels
    ax6.text(0.02, 0.98, 'Question Mark', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='top')
    ax6.text(0.98, 0.98, 'Star', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='top', ha='right')
    ax6.text(0.02, 0.02, 'Dog', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='bottom')
    ax6.text(0.98, 0.02, 'Cash Cow', transform=ax6.transAxes,
            fontsize=12, fontweight='bold', va='bottom', ha='right')

    ax6.set_xlabel('Market Share (%)', fontsize=13, fontweight='bold')
    ax6.set_ylabel('CAGR (%)', fontsize=13, fontweight='bold')
    ax6.set_title('BCG Portfolio Matrix - Oman Exports to UAE',
                 fontsize=16, fontweight='bold', pad=20)
    ax6.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=True, shadow=True)
    ax6.grid(True, alpha=0.3)

    plt.tight_layout()
    fig6.savefig('/tmp/fig6_bcg_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Figure 6: BCG Matrix")

# ========================================================================
# FIGURE 7: YEAR-OVER-YEAR GROWTH RATES
# ========================================================================
fig7, ax7 = plt.subplots(figsize=(14, 8))

yoy_exp = df_trade['Exports'].pct_change() * 100
yoy_imp = df_trade['Imports'].pct_change() * 100

ax7.plot(years_list[1:], yoy_exp.iloc[1:], marker='o', linewidth=2.5,
         markersize=8, label='Exports YoY%', color='#2E86AB')
ax7.plot(years_list[1:], yoy_imp.iloc[1:], marker='^', linewidth=2.5,
         markersize=8, label='Imports YoY%', color='#F18F01')
ax7.axhline(y=0, color='black', linestyle='--', linewidth=1.5)

ax7.set_xlabel('Year', fontsize=13, fontweight='bold')
ax7.set_ylabel('Year-over-Year Growth (%)', fontsize=13, fontweight='bold')
ax7.set_title('Annual Growth Rate Dynamics', fontsize=16, fontweight='bold', pad=20)
ax7.legend(loc='best', frameon=True, shadow=True, fontsize=11)
ax7.grid(True, alpha=0.3)
ax7.set_xticks(years_list[1:])

plt.tight_layout()
fig7.savefig('/tmp/fig7_yoy_growth.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Figure 7: YoY Growth Rates")

print("\n✅ ALL VISUALIZATIONS GENERATED!")
print("="*80)


📊 GENERATING COMPREHENSIVE VISUALIZATIONS
✅ Figure 1: Trade Flow Trends
✅ Figure 2: Trade Balance Evolution
✅ Figure 3: Coverage Ratio Trend
✅ Figure 4: CAGR Comparison
✅ Figure 5: Export Composition
✅ Figure 6: BCG Matrix
✅ Figure 7: YoY Growth Rates

✅ ALL VISUALIZATIONS GENERATED!


In [14]:
# ============================================================================
# CELL 10: GENERATE COMPREHENSIVE WORD DOCUMENT
# ============================================================================

print("\n📄 GENERATING COMPREHENSIVE WORD DOCUMENT REPORT")
print("="*80)

REPORTING_COUNTRY = 'Oman'
PARTNER_COUNTRY = 'UAE'

doc = Document()

# ========================================================================
# CONFIGURE STYLES
# ========================================================================
styles = doc.styles

# Title style
title_style = styles['Title']
title_style.font.size = Pt(26)
title_style.font.bold = True
title_style.font.color.rgb = RGBColor(31, 78, 120)

# Heading styles
h1_style = styles['Heading 1']
h1_style.font.size = Pt(18)
h1_style.font.bold = True
h1_style.font.color.rgb = RGBColor(31, 78, 120)

h2_style = styles['Heading 2']
h2_style.font.size = Pt(15)
h2_style.font.bold = True
h2_style.font.color.rgb = RGBColor(46, 116, 181)

# ========================================================================
# TITLE PAGE
# ========================================================================
print("  Creating title page...")

title = doc.add_heading(f'{REPORTING_COUNTRY}-{PARTNER_COUNTRY}', level=0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

subtitle = doc.add_heading('Comprehensive Bilateral Trade Dynamics Report', level=2)
subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER

subtitle2 = doc.add_heading('Strategic Assessment & Portfolio Analysis', level=3)
subtitle2.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()
period = doc.add_paragraph(f'Analysis Period: 2015-2023')
period.alignment = WD_ALIGN_PARAGRAPH.CENTER
period.runs[0].font.size = Pt(12)

date_para = doc.add_paragraph(f'Report Date: {datetime.now().strftime("%B %d, %Y")}')
date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
date_para.runs[0].font.size = Pt(12)

doc.add_paragraph()
doc.add_paragraph()
doc.add_paragraph()

prepared = doc.add_paragraph('Prepared by:')
prepared.alignment = WD_ALIGN_PARAGRAPH.CENTER
prepared.runs[0].font.size = Pt(11)

company = doc.add_paragraph('Deloitte Consulting - GCC Trade Analytics Division')
company.alignment = WD_ALIGN_PARAGRAPH.CENTER
company.runs[0].font.bold = True
company.runs[0].font.size = Pt(12)

doc.add_page_break()

# ========================================================================
# EXECUTIVE SUMMARY
# ========================================================================
print("  Creating executive summary...")

doc.add_heading('Executive Summary', level=1)

# Narrative
exec_narrative = f"""
The bilateral trade relationship between {REPORTING_COUNTRY} and {PARTNER_COUNTRY} over the nine-year period (2015-2023)
generated a cumulative trade volume of ${cum_volume/1e9:.1f} billion USD, with a cumulative trade balance of
${cum_balance/1e9:.1f} billion USD. The relationship is characterized by a structural trade imbalance, recording
deficits in {deficit_years} out of {years} years analyzed.

Key characteristics include:
- Trade volume grew at {cagr_volume:.1f}% CAGR, indicating {'robust' if cagr_volume > 5 else 'moderate'} bilateral economic integration
- Exports expanded at {cagr_exports:.1f}% CAGR, {'outpacing' if cagr_exports > cagr_imports else 'lagging'} import growth ({cagr_imports:.1f}% CAGR)
- Export coverage averaged {avg_coverage:.1f}%, reflecting {'significant' if avg_coverage < 60 else 'moderate'} import dependency
- Re-exports constitute {reexport_share:.1f}% of total exports, underscoring {REPORTING_COUNTRY}'s {'significant' if reexport_share > 20 else 'limited'} role as a regional trade hub
"""

doc.add_paragraph(exec_narrative)

# Key Metrics Table
doc.add_heading('Key Performance Indicators', level=2)

metrics_table = doc.add_table(rows=1, cols=2)
metrics_table.style = 'Light Grid Accent 1'
hdr = metrics_table.rows[0].cells
hdr[0].text = 'Metric'
hdr[1].text = 'Value'
hdr[0].paragraphs[0].runs[0].font.bold = True
hdr[1].paragraphs[0].runs[0].font.bold = True

key_metrics = [
    ('Cumulative Trade Volume (2015-2023)', f'${cum_volume/1e9:.1f}B'),
    ('Cumulative Trade Balance', f'${cum_balance/1e9:.1f}B'),
    ('Trade Volume CAGR', f'{cagr_volume:.1f}%'),
    ('Export CAGR', f'{cagr_exports:.1f}%'),
    ('Import CAGR', f'{cagr_imports:.1f}%'),
    ('Re-export CAGR', f'{cagr_reexports:.1f}%'),
    ('Latest Year Coverage Ratio', f'{latest_coverage:.1f}%'),
    ('Average Coverage Ratio', f'{avg_coverage:.1f}%'),
    ('Deficit Years', f'{deficit_years} out of {years}'),
    ('Peak Deficit', f'${abs(peak_deficit)/1e9:.1f}B'),
    ('Re-export Share of Total Exports', f'{reexport_share:.1f}%'),
    ('Export Volatility (Std Dev)', f'{vol_exports:.1f}%'),
    ('Import Volatility (Std Dev)', f'{vol_imports:.1f}%'),
]

if has_product_data and hhi_exports > 0:
    key_metrics.extend([
        ('Export HHI Concentration Index', f'{hhi_exports:.0f}'),
        ('Import HHI Concentration Index', f'{hhi_imports:.0f}'),
    ])

for metric, value in key_metrics:
    row = metrics_table.add_row().cells
    row[0].text = metric
    row[1].text = value

doc.add_page_break()

# ========================================================================
# SECTION 1: TRADE BALANCE ANALYSIS
# ========================================================================
print("  Creating Section 1: Trade Balance Analysis...")

doc.add_heading('1. Trade Balance Analysis and Structural Assessment', level=1)

balance_text = f"""
The bilateral trade relationship exhibits a persistent deficit pattern, with cumulative trade balance of
${cum_balance/1e9:.1f}B over the {years}-year period. This structural imbalance reflects fundamental asymmetries
in the economic relationship, with {PARTNER_COUNTRY} serving as a major supplier to the {REPORTING_COUNTRY} market
while {REPORTING_COUNTRY} faces challenges in penetrating the {PARTNER_COUNTRY} market at comparable scale.
"""
doc.add_paragraph(balance_text)

doc.add_heading('1.1 Annual Trade Balance Dynamics', level=2)

# Annual balance table
balance_table = doc.add_table(rows=1, cols=7)
balance_table.style = 'Light Grid Accent 1'
hdr = balance_table.rows[0].cells
headers = ['Year', 'Exports\n($M)', 'Re-exports\n($M)', 'Imports\n($M)',
           'Total Exports\n($M)', 'Balance\n($M)', 'Coverage\n(%)']
for i, h in enumerate(headers):
    hdr[i].text = h
    hdr[i].paragraphs[0].runs[0].font.bold = True

for year, row_data in df_trade.iterrows():
    cells = balance_table.add_row().cells
    cells[0].text = str(year)
    cells[1].text = f"{row_data['Exports']/1e6:,.0f}"
    cells[2].text = f"{row_data['Re-exports']/1e6:,.0f}"
    cells[3].text = f"{row_data['Imports']/1e6:,.0f}"
    cells[4].text = f"{row_data['Total_Exports']/1e6:,.0f}"
    cells[5].text = f"{row_data['Trade_Balance']/1e6:,.0f}"
    cells[6].text = f"{row_data['Coverage_Ratio']:.1f}"

doc.add_paragraph()

# Key insights
doc.add_heading('1.2 Key Balance Insights', level=2)

insights = [
    f"Persistent deficit pattern: {deficit_years} out of {years} years recorded trade deficits",
    f"Peak deficit of ${abs(peak_deficit)/1e9:.1f}B occurred in {df_trade['Trade_Balance'].idxmin()}",
    f"{'Improving trend' if df_trade['Trade_Balance'].iloc[-1] > df_trade['Trade_Balance'].iloc[0] else 'Widening deficit'} observed over the analysis period",
    f"Cumulative deficit represents {abs(cum_balance)/cum_volume*100:.1f}% of total trade volume",
]

for insight in insights:
    p = doc.add_paragraph(insight, style='List Bullet')

doc.add_paragraph()
doc.add_heading('1.3 Trade Flow Visualization', level=2)
doc.add_picture('/tmp/fig1_trade_flows.png', width=Inches(6.5))
doc.add_paragraph('Figure 1: Trade flow trends showing exports, re-exports, and imports with linear trend lines (2015-2023).',
                 style='Caption')

doc.add_page_break()

# ========================================================================
# SECTION 2: GROWTH DYNAMICS
# ========================================================================
print("  Creating Section 2: Growth Dynamics...")

doc.add_heading('2. Growth Dynamics and Time Series Analysis', level=1)

growth_narrative = f"""
The bilateral trade relationship demonstrates dynamic growth patterns with differentiated trajectories across trade components.
Direct exports from {REPORTING_COUNTRY} to {PARTNER_COUNTRY} grew at {cagr_exports:.1f}% CAGR,
{'demonstrating expanding market penetration and competitive positioning' if cagr_exports > 0 else 'indicating market share challenges and competitive pressures'}.
Re-exports contributed {reexport_share:.1f}% of total export value, growing at {cagr_reexports:.1f}% CAGR, which underscores
{REPORTING_COUNTRY}'s {'significant' if reexport_share > 20 else 'limited'} role as a regional entrepôt center.

Import growth at {cagr_imports:.1f}% CAGR {'exceeded export expansion' if cagr_imports > cagr_exports else 'trailed export growth'},
contributing to {'widening' if cagr_imports > cagr_exports else 'narrowing'} trade imbalances. The overall trade volume expanded at
{cagr_volume:.1f}% CAGR, reflecting {'deepening' if cagr_volume > 5 else 'moderate'} economic integration within the GCC framework.
"""
doc.add_paragraph(growth_narrative)

doc.add_heading('2.1 Compound Annual Growth Rates', level=2)
doc.add_picture('/tmp/fig4_cagr.png', width=Inches(6.5))
doc.add_paragraph('Figure 2: CAGR comparison across all trade components (2015-2023).', style='Caption')

doc.add_paragraph()
doc.add_heading('2.2 Trend Analysis and Statistical Significance', level=2)

trend_text = f"""
Linear regression analysis reveals:
- Export trend R² = {export_r2:.3f} ({'strong' if export_r2 > 0.7 else 'moderate' if export_r2 > 0.4 else 'weak'} trend fit)
- Import trend R² = {import_r2:.3f} ({'strong' if import_r2 > 0.7 else 'moderate' if import_r2 > 0.4 else 'weak'} trend fit)

The {'high' if max(export_r2, import_r2) > 0.7 else 'moderate'} R² values indicate {'predictable' if max(export_r2, import_r2) > 0.7 else 'somewhat volatile'}
trade patterns with {'minimal' if max(export_r2, import_r2) > 0.7 else 'notable'} deviation from linear trends.
"""
doc.add_paragraph(trend_text)

doc.add_heading('2.3 Volatility Assessment', level=2)

# Volatility table
vol_table = doc.add_table(rows=1, cols=4)
vol_table.style = 'Light Grid Accent 1'
vol_hdr = vol_table.rows[0].cells
vol_headers = ['Component', 'YoY Volatility (%)', 'Assessment', 'Interpretation']
for i, h in enumerate(vol_headers):
    vol_hdr[i].text = h
    vol_hdr[i].paragraphs[0].runs[0].font.bold = True

vol_data = [
    ('Exports', vol_exports,
     'High' if vol_exports > 20 else 'Moderate' if vol_exports > 10 else 'Low',
     'Significant fluctuations' if vol_exports > 20 else 'Stable growth' if vol_exports < 10 else 'Some variability'),
    ('Imports', vol_imports,
     'High' if vol_imports > 20 else 'Moderate' if vol_imports > 10 else 'Low',
     'Significant fluctuations' if vol_imports > 20 else 'Stable growth' if vol_imports < 10 else 'Some variability'),
]

for comp, vol, assess, interp in vol_data:
    row = vol_table.add_row().cells
    row[0].text = comp
    row[1].text = f'{vol:.1f}%'
    row[2].text = assess
    row[3].text = interp

doc.add_paragraph()
doc.add_heading('2.4 Year-over-Year Growth Dynamics', level=2)
doc.add_picture('/tmp/fig7_yoy_growth.png', width=Inches(6.5))
doc.add_paragraph('Figure 3: Annual growth rate fluctuations showing volatility patterns.', style='Caption')

doc.add_page_break()

# ========================================================================
# SECTION 3: TRADE BALANCE DECOMPOSITION
# ========================================================================
print("  Creating Section 3: Trade Balance Decomposition...")

doc.add_heading('3. Trade Balance Decomposition and Coverage Analysis', level=1)

doc.add_heading('3.1 Trade Balance Evolution', level=2)
doc.add_picture('/tmp/fig2_balance.png', width=Inches(6.5))
doc.add_paragraph('Figure 4: Annual trade balance showing persistent deficit pattern with value annotations.', style='Caption')

doc.add_paragraph()

deficit_analysis = f"""
The trade balance exhibits a {'persistent' if deficit_years >= years * 0.7 else 'intermittent'} deficit pattern.
The cumulative deficit of ${abs(cum_balance)/1e9:.1f}B represents {abs(cum_balance)/cum_volume*100:.1f}% of total bilateral trade volume,
indicating structural imbalances rather than cyclical fluctuations.

Key observations:
- Average annual deficit: ${abs(df_trade['Trade_Balance'].mean())/1e9:.1f}B
- Peak deficit year: {df_trade['Trade_Balance'].idxmin()} (${abs(peak_deficit)/1e9:.1f}B)
- {'Deteriorating' if df_trade['Trade_Balance'].iloc[-1] < df_trade['Trade_Balance'].iloc[0] else 'Improving'} trend in recent years
"""
doc.add_paragraph(deficit_analysis)

doc.add_heading('3.2 Export Coverage of Imports', level=2)
doc.add_picture('/tmp/fig3_coverage.png', width=Inches(6.5))
doc.add_paragraph('Figure 5: Export coverage ratio showing the extent of import dependency.', style='Caption')

doc.add_paragraph()

coverage_analysis = f"""
The export coverage ratio averaged {avg_coverage:.1f}% over the analysis period, with the latest year at {latest_coverage:.1f}%.
This indicates that {REPORTING_COUNTRY}'s total exports (including re-exports) cover only {latest_coverage:.0f}% of its import bill
from {PARTNER_COUNTRY}, reflecting {'high' if latest_coverage < 50 else 'moderate' if latest_coverage < 75 else 'low'} import dependency.

Coverage ratio insights:
- Minimum coverage: {df_trade['Coverage_Ratio'].min():.1f}% in {df_trade['Coverage_Ratio'].idxmin()}
- Maximum coverage: {df_trade['Coverage_Ratio'].max():.1f}% in {df_trade['Coverage_Ratio'].idxmax()}
- {'Improving' if df_trade['Coverage_Ratio'].iloc[-1] > df_trade['Coverage_Ratio'].iloc[0] else 'Declining'} trend observed
- {latest_coverage:.0f}% coverage implies ${(100-latest_coverage)/100 * df_trade['Imports'].iloc[-1]/1e9:.1f}B uncovered import demand
"""
doc.add_paragraph(coverage_analysis)

doc.add_heading('3.3 Re-export Trade Dynamics', level=2)
doc.add_picture('/tmp/fig5_composition.png', width=Inches(6.5))
doc.add_paragraph('Figure 6: Export composition showing the contribution of direct exports versus re-exports.', style='Caption')

doc.add_paragraph()

reexport_analysis = f"""
Re-exports constitute {reexport_share:.1f}% of total exports to {PARTNER_COUNTRY}, growing at {cagr_reexports:.1f}% CAGR.
This {'substantial' if reexport_share > 20 else 'moderate' if reexport_share > 10 else 'limited'} re-export activity underscores
{REPORTING_COUNTRY}'s role as {'a significant regional trade hub' if reexport_share > 20 else 'a limited entrepôt center'} for goods
destined to {PARTNER_COUNTRY}.

Re-export trends:
- Average re-export share: {(df_trade['Re-exports'] / df_trade['Total_Exports'] * 100).mean():.1f}%
- Re-export peak year: {df_trade['Re-exports'].idxmax()} (${df_trade['Re-exports'].max()/1e9:.1f}B)
- {'Increasing' if df_trade['Re-exports'].iloc[-1] > df_trade['Re-exports'].iloc[0] else 'Decreasing'} re-export activity over time
"""
doc.add_paragraph(reexport_analysis)

doc.add_page_break()

# ========================================================================
# SECTION 4: PRODUCT PORTFOLIO ANALYSIS (IF AVAILABLE)
# ========================================================================
if has_product_data and not bcg_export.empty:
    print("  Creating Section 4: Product Portfolio Analysis...")

    doc.add_heading('4. Product Portfolio and Concentration Analysis', level=1)

    doc.add_heading('4.1 BCG Matrix Portfolio Assessment', level=2)

    bcg_narrative = f"""
The BCG Matrix analysis categorizes {len(bcg_export)} export products based on market share and growth rates,
providing strategic insights for portfolio optimization and resource allocation.
"""
    doc.add_paragraph(bcg_narrative)

    # BCG Summary Table
    bcg_table = doc.add_table(rows=1, cols=4)
    bcg_table.style = 'Light Grid Accent 1'
    bcg_hdr = bcg_table.rows[0].cells
    bcg_headers = ['Category', 'Product Count', 'Total Value ($B)', 'Avg CAGR (%)']
    for i, h in enumerate(bcg_headers):
        bcg_hdr[i].text = h
        bcg_hdr[i].paragraphs[0].runs[0].font.bold = True

    if not bcg_summary.empty:
        for category in ['Star', 'Cash Cow', 'Question Mark', 'Dog']:
            if category in bcg_summary.index:
                row = bcg_table.add_row().cells
                row[0].text = category
                row[1].text = f"{bcg_summary.loc[category, 'Product']:.0f}"
                row[2].text = f"${bcg_summary.loc[category, 'Total_Value']/1e9:.2f}"
                row[3].text = f"{bcg_summary.loc[category, 'CAGR']:.1f}%"

    doc.add_paragraph()
    doc.add_picture('/tmp/fig6_bcg_matrix.png', width=Inches(6.5))
    doc.add_paragraph('Figure 7: BCG Portfolio Matrix showing product distribution across strategic categories.', style='Caption')

    doc.add_paragraph()
    doc.add_heading('4.2 Star Products (High Growth, High Share)', level=2)

    if 'Star' in bcg_export['BCG_Category'].values:
        stars = bcg_export[bcg_export['BCG_Category'] == 'Star'].nlargest(10, 'Total_Value')

        star_table = doc.add_table(rows=1, cols=4)
        star_table.style = 'Light Grid Accent 1'
        star_hdr = star_table.rows[0].cells
        star_headers = ['Product', 'CAGR (%)', 'Market Share (%)', 'Value ($M)']
        for i, h in enumerate(star_headers):
            star_hdr[i].text = h
            star_hdr[i].paragraphs[0].runs[0].font.bold = True

        for idx, row in stars.head(10).iterrows():
            cells = star_table.add_row().cells
            cells[0].text = str(row['Product'])[:50]
            cells[1].text = f"{row['CAGR']:.1f}"
            cells[2].text = f"{row['Market_Share']:.2f}"
            cells[3].text = f"${row['Total_Value']/1e6:.1f}"

        doc.add_paragraph()
        star_text = f"""
Star products represent high-growth, high-market-share items requiring continued investment to maintain competitive advantage.
These {len(stars)} products collectively account for ${stars['Total_Value'].sum()/1e9:.2f}B in export value.
"""
        doc.add_paragraph(star_text)

    doc.add_heading('4.3 Concentration Analysis (HHI)', level=2)

    hhi_text = f"""
The Herfindahl-Hirschman Index (HHI) measures market concentration:

Export Market Concentration:
- HHI: {hhi_exports:.0f}
- Assessment: {'Low concentration (competitive market)' if hhi_exports < 1500 else 'Moderate concentration' if hhi_exports < 2500 else 'High concentration (oligopolistic)'}
- Top 10 products control {bcg_export.nlargest(10, 'Market_Share')['Market_Share'].sum():.1f}% of export value

Import Market Concentration:
- HHI: {hhi_imports:.0f}
- Assessment: {'Low concentration (competitive market)' if hhi_imports < 1500 else 'Moderate concentration' if hhi_imports < 2500 else 'High concentration (oligopolistic)'}
- Top 10 products control {bcg_import.nlargest(10, 'Market_Share')['Market_Share'].sum():.1f}% of import value

{'High concentration suggests vulnerability to supply disruptions and limited diversification' if max(hhi_exports, hhi_imports) > 2500 else 'Moderate concentration indicates balanced portfolio with manageable concentration risk' if max(hhi_exports, hhi_imports) > 1500 else 'Low concentration reflects a well-diversified trade portfolio'}.
"""
    doc.add_paragraph(hhi_text)

    doc.add_heading('4.4 Top Products by Value', level=2)

    # Top exports table
    top_exp_table = doc.add_table(rows=1, cols=3)
    top_exp_table.style = 'Light Grid Accent 1'
    top_exp_hdr = top_exp_table.rows[0].cells
    top_exp_hdr[0].text = 'Top Export Products'
    top_exp_hdr[1].text = 'Market Share (%)'
    top_exp_hdr[2].text = 'Value ($M)'
    for cell in top_exp_hdr:
        cell.paragraphs[0].runs[0].font.bold = True

    top_exports = bcg_export.nlargest(10, 'Market_Share')
    for idx, row in top_exports.iterrows():
        cells = top_exp_table.add_row().cells
        cells[0].text = str(row['Product'])[:60]
        cells[1].text = f"{row['Market_Share']:.2f}%"
        cells[2].text = f"${row['Total_Value']/1e6:.1f}"

    doc.add_page_break()

# ========================================================================
# SECTION 5: STRATEGIC RECOMMENDATIONS
# ========================================================================
print("  Creating Section 5: Strategic Recommendations...")

doc.add_heading('4. Strategic Recommendations and Policy Priorities' if not has_product_data else '5. Strategic Recommendations and Policy Priorities', level=1)

strategic_intro = f"""
Given the structural characteristics of the bilateral trade relationship—persistent deficits, {'high' if latest_coverage < 50 else 'moderate'} import dependency,
and {cagr_volume:.1f}% trade volume growth—{REPORTING_COUNTRY} should prioritize a multi-faceted strategy encompassing export expansion,
import substitution, and trade facilitation initiatives.
"""
doc.add_paragraph(strategic_intro)

doc.add_heading('4.1 Export Expansion Strategies' if not has_product_data else '5.1 Export Expansion Strategies', level=2)

if cagr_exports < cagr_imports:
    export_intro = f"""
Given that exports are growing slower than imports ({cagr_exports:.1f}% vs {cagr_imports:.1f}% CAGR), {REPORTING_COUNTRY} must
aggressively pursue export expansion through:
"""
else:
    export_intro = f"""
While exports are growing faster than imports ({cagr_exports:.1f}% vs {cagr_imports:.1f}% CAGR), {REPORTING_COUNTRY} should
consolidate gains and accelerate market penetration through:
"""
doc.add_paragraph(export_intro)

export_strategies = [
    f"Market Diversification: Expand beyond current product concentration (HHI: {hhi_exports:.0f}) to reduce vulnerability to demand shocks",
    f"Value Chain Upgrading: Move into higher value-added products to improve export unit values and margins",
    "Trade Facilitation: Reduce transaction costs and time-to-market through customs modernization and logistics optimization",
    f"Strategic Partnerships: Forge alliances with key distributors and importers in {PARTNER_COUNTRY} to enhance market access",
    "Export Financing: Establish dedicated credit facilities and export credit insurance to support SME exporters",
    "Quality Certification: Fast-track mutual recognition agreements for standards and certifications to reduce market entry barriers",
    f"Sector Focus: {'Target Star products for investment and Question Marks for growth acceleration' if has_product_data else 'Identify high-potential sectors through market research'}",
]

for strategy in export_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_heading('4.2 Import Substitution and Localization' if not has_product_data else '5.2 Import Substitution and Localization', level=2)

import_sub_intro = f"""
With imports at ${df_trade['Imports'].iloc[-1]/1e9:.1f}B annually (latest year) and growing at {cagr_imports:.1f}% CAGR,
strategic import substitution opportunities should be evaluated in sectors where:
"""
doc.add_paragraph(import_sub_intro)

import_sub_strategies = [
    "Domestic production capacity can be developed cost-effectively without sacrificing quality",
    "Technology transfer and know-how can be acquired through FDI or licensing arrangements",
    "Import dependency creates strategic vulnerabilities (critical inputs, defense-related items)",
    "Regional content requirements align with GCC common market integration objectives",
    "Local value addition can create employment and enhance industrial capabilities",
    f"{'High-concentration imports (HHI: {:.0f}) present substitution opportunities'.format(hhi_imports) if has_product_data else 'Import concentration analysis identifies priority substitution targets'}",
]

for strategy in import_sub_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_heading('4.3 Trade Facilitation and Logistics Optimization' if not has_product_data else '5.3 Trade Facilitation and Logistics Optimization', level=2)

facilitation_intro = f"""
Given the substantial trade volumes (${cum_volume/1e9:.1f}B cumulative) and growth dynamics, both countries should prioritize:
"""
doc.add_paragraph(facilitation_intro)

facilitation_strategies = [
    "Harmonization of customs procedures and documentation requirements to reduce clearance times",
    "Investment in cross-border logistics infrastructure, including cold chain facilities for perishables",
    "Digital trade platforms for streamlined processing, real-time tracking, and electronic documentation",
    "Authorized Economic Operator (AEO) mutual recognition programs for trusted trader benefits",
    "Reduction of non-tariff barriers through regulatory cooperation and technical standards alignment",
    "Single Window implementation for integrated government agency clearances",
    "Risk-based inspection protocols to expedite low-risk shipments while maintaining security",
]

for strategy in facilitation_strategies:
    doc.add_paragraph(strategy, style='List Bullet')

doc.add_page_break()

# ========================================================================
# SECTION 6: IMPLEMENTATION ROADMAP
# ========================================================================
print("  Creating Section 6: Implementation Roadmap...")

doc.add_heading('5. Implementation Roadmap and Action Plan' if not has_product_data else '6. Implementation Roadmap and Action Plan', level=1)

roadmap_intro = f"""
Successful execution of the strategic opportunities requires coordinated action across government agencies, trade promotion bodies,
and private sector stakeholders. The following phased approach prioritizes quick wins while building toward long-term structural transformation.
"""
doc.add_paragraph(roadmap_intro)

doc.add_heading('Phase 1: Immediate Actions (0-6 months)', level=2)

immediate_actions = [
    "Establish bilateral trade committee with quarterly review mechanism and senior-level representation",
    f"Commission detailed market study of high-potential sectors in {PARTNER_COUNTRY}, leveraging product-level trade data",
    "Launch trade mission with B2B matchmaking events to strengthen commercial linkages",
    "Fast-track mutual recognition agreements for product standards, certifications, and testing",
    "Create dedicated export financing facility for SMEs with competitive rates and simplified procedures",
    f"{'Initiate targeted campaigns for Star products identified in BCG analysis' if has_product_data else 'Identify and promote high-growth export products'}",
    "Conduct import substitution feasibility studies for strategic sectors",
]

for i, action in enumerate(immediate_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_heading('Phase 2: Medium-term Initiatives (6-18 months)', level=2)

medium_actions = [
    "Negotiate preferential market access arrangements for priority products under GCC framework",
    f"Establish permanent trade office in {PARTNER_COUNTRY} or strengthen existing commercial representation",
    f"Launch targeted marketing campaigns {'for Star and Question Mark products' if has_product_data else 'in high-potential product categories'}",
    "Develop supplier capacity building programs to meet export quality standards and certifications",
    "Implement comprehensive trade information portal with real-time market intelligence and logistics tracking",
    "Establish logistics corridor with enhanced infrastructure and streamlined border procedures",
    "Create export consortia for SMEs to achieve economies of scale in marketing and distribution",
    "Initiate technology transfer partnerships in import substitution priority sectors",
]

for i, action in enumerate(medium_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_heading('Phase 3: Long-term Strategic Initiatives (18+ months)', level=2)

long_actions = [
    "Institutionalize bilateral trade monitoring framework with KPIs and annual assessments",
    "Develop joint industrial cooperation projects in strategic sectors (manufacturing, logistics, services)",
    "Establish integrated logistics and distribution hubs to enhance regional connectivity",
    "Create innovation partnerships and R&D collaborations in high-tech sectors",
    "Implement comprehensive trade facilitation reforms aligned with WTO Trade Facilitation Agreement",
    "Develop free trade zone or special economic zone targeting bilateral trade facilitation",
    f"{'Optimize portfolio mix by divesting from Dog products and reinvesting in Stars' if has_product_data else 'Achieve balanced export portfolio with reduced concentration'}",
    "Achieve sustainable trade balance improvement with coverage ratio target of 75%+",
]

for i, action in enumerate(long_actions, 1):
    doc.add_paragraph(f"{action}", style='List Number')

doc.add_page_break()

# ========================================================================
# SECTION 7: CONCLUSION
# ========================================================================
print("  Creating Section 7: Conclusion...")

doc.add_heading('6. Conclusion and Forward Outlook' if not has_product_data else '7. Conclusion and Forward Outlook', level=1)

conclusion_text = f"""
The bilateral trade relationship between {REPORTING_COUNTRY} and {PARTNER_COUNTRY} presents a complex picture of
deepening economic integration alongside persistent structural imbalances. The analysis reveals:

Strengths:
- Robust trade volume growth ({cagr_volume:.1f}% CAGR) demonstrating strong bilateral economic ties
- {'Dynamic export expansion ({:.1f}% CAGR) showing competitive gains'.format(cagr_exports) if cagr_exports > 0 else 'Export base with potential for acceleration'}
- {'Significant re-export activity ({:.1f}% of total exports) leveraging regional hub positioning'.format(reexport_share) if reexport_share > 15 else 'Foundation for developing regional trade hub capabilities'}
- {'Well-diversified product portfolio (HHI: {:.0f})'.format(hhi_exports) if has_product_data and hhi_exports < 1500 else 'Established trade infrastructure and relationships'}

Challenges:
- Persistent trade deficit (${abs(cum_balance)/1e9:.1f}B cumulative) requiring strategic rebalancing
- {'High' if latest_coverage < 50 else 'Moderate'} import dependency (coverage ratio: {latest_coverage:.1f}%)
- {'Import growth ({:.1f}% CAGR) outpacing exports'.format(cagr_imports) if cagr_imports > cagr_exports else 'Need to sustain export momentum'}
- {'High product concentration (HHI: {:.0f}) creating vulnerability'.format(hhi_exports) if has_product_data and hhi_exports > 2500 else 'Room for export diversification'}

The strategic recommendations and implementation roadmap outlined in this report provide a comprehensive framework for addressing
these challenges while capitalizing on opportunities. Success will require sustained commitment from government agencies, trade
promotion bodies, and the private sector, working in coordination to transform the bilateral trade relationship into a more balanced
and mutually beneficial partnership.

With the GCC economic integration framework providing institutional support, and both countries' Vision 2030+ strategies emphasizing
economic diversification and trade expansion, the conditions are favorable for meaningful progress. The key is translating strategic
intent into concrete actions, with clear accountability, adequate resourcing, and regular monitoring of progress against defined KPIs.
"""
doc.add_paragraph(conclusion_text)

doc.add_paragraph()
doc.add_paragraph()

# Disclaimer
disclaimer = doc.add_paragraph(
    "DISCLAIMER: This report is based on official trade statistics and economic analysis. The strategic recommendations are "
    "provided for informational and planning purposes. Actual policy implementation should be informed by additional stakeholder "
    "consultations, detailed feasibility studies, cost-benefit analysis, and consideration of broader economic and political factors. "
    "The authors and Deloitte Consulting assume no liability for decisions made based on this analysis."
)
disclaimer.runs[0].font.size = Pt(9)
disclaimer.runs[0].font.italic = True
disclaimer.runs[0].font.color.rgb = RGBColor(100, 100, 100)

# ========================================================================
# SAVE DOCUMENT
# ========================================================================
print("\n  Saving document...")

output_filename = f'{REPORTING_COUNTRY}_{PARTNER_COUNTRY}_Comprehensive_Trade_Report_{datetime.now().strftime("%Y%m%d")}.docx'
doc.save(f'/content/{output_filename}')

print(f"\n{'='*80}")
print(f"✅ COMPREHENSIVE REPORT GENERATED SUCCESSFULLY!")
print(f"{'='*80}")
print(f"\n📄 Filename: {output_filename}")
print(f"📊 Sections: {6 if not has_product_data else 7}")
print(f"📈 Figures: {7 if has_product_data else 5}")
print(f"📋 Tables: {8 if has_product_data else 4}")
print(f"📄 Estimated Pages: {'20-25' if has_product_data else '15-18'}")
print(f"\nReport includes:")
print(f"  ✓ Executive Summary with 13+ KPIs")
print(f"  ✓ Trade Balance Analysis (annual table)")
print(f"  ✓ Growth Dynamics (CAGR, volatility, trends)")
print(f"  ✓ Coverage Analysis (import dependency)")
print(f"  ✓ Re-export Dynamics")
if has_product_data:
    print(f"  ✓ BCG Portfolio Matrix")
    print(f"  ✓ HHI Concentration Analysis")
    print(f"  ✓ Top Products Tables")
print(f"  ✓ Strategic Recommendations (15+ actions)")
print(f"  ✓ 3-Phase Implementation Roadmap")
print(f"  ✓ 7 Professional Visualizations")
print(f"\n{'='*80}")


📄 GENERATING COMPREHENSIVE WORD DOCUMENT REPORT
  Creating title page...
  Creating executive summary...
  Creating Section 1: Trade Balance Analysis...
  Creating Section 2: Growth Dynamics...
  Creating Section 3: Trade Balance Decomposition...
  Creating Section 4: Product Portfolio Analysis...
  Creating Section 5: Strategic Recommendations...
  Creating Section 6: Implementation Roadmap...
  Creating Section 7: Conclusion...

  Saving document...

✅ COMPREHENSIVE REPORT GENERATED SUCCESSFULLY!

📄 Filename: Oman_UAE_Comprehensive_Trade_Report_20251207.docx
📊 Sections: 7
📈 Figures: 7
📋 Tables: 8
📄 Estimated Pages: 20-25

Report includes:
  ✓ Executive Summary with 13+ KPIs
  ✓ Trade Balance Analysis (annual table)
  ✓ Growth Dynamics (CAGR, volatility, trends)
  ✓ Coverage Analysis (import dependency)
  ✓ Re-export Dynamics
  ✓ BCG Portfolio Matrix
  ✓ HHI Concentration Analysis
  ✓ Top Products Tables
  ✓ Strategic Recommendations (15+ actions)
  ✓ 3-Phase Implementation Roadmap
 

In [15]:
# ============================================================================
# CELL 11: DOWNLOAD THE COMPREHENSIVE REPORT
# ============================================================================

from google.colab import files
import os

print("\n" + "="*80)
print("📥 DOWNLOADING YOUR COMPREHENSIVE REPORT")
print("="*80)

# List all generated files
print("\n📁 Generated files:")
for f in os.listdir('/content'):
    if f.endswith('.docx') and 'Comprehensive_Trade_Report' in f:
        size = os.path.getsize(f'/content/{f}')
        print(f"  ✓ {f} ({size:,} bytes)")

# Download the report
if os.path.exists(f'/content/{output_filename}'):
    print(f"\n🔽 Downloading: {output_filename}")
    files.download(f'/content/{output_filename}')
    print("\n✅ DOWNLOAD COMPLETE!")
    print("\nYour comprehensive Word document is now saved to your PC!")
    print("\n" + "="*80)
    print("🎉 ANALYSIS COMPLETE - ENJOY YOUR REPORT!")
    print("="*80)
else:
    print("\n❌ Report file not found!")

print("\n📊 REPORT CONTENTS:")
print("   • Executive Summary with complete KPI table")
print("   • Trade Balance Analysis with annual data table")
print("   • Growth Dynamics with CAGR charts")
print("   • Coverage Analysis with trend visualization")
print("   • Volatility Assessment")
print("   • Re-export Dynamics")
if has_product_data:
    print("   • BCG Portfolio Matrix with product classification")
    print("   • HHI Concentration Analysis")
    print("   • Top Products Tables (Stars, Top 10 exports/imports)")
print("   • Strategic Recommendations (15+ actionable strategies)")
print("   • 3-Phase Implementation Roadmap (20+ specific actions)")
print("   • Professional charts embedded throughout")
print("\n" + "="*80)


📥 DOWNLOADING YOUR COMPREHENSIVE REPORT

📁 Generated files:
  ✓ Oman_UAE_Comprehensive_Trade_Report_20251207.docx (1,566,357 bytes)

🔽 Downloading: Oman_UAE_Comprehensive_Trade_Report_20251207.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ DOWNLOAD COMPLETE!

Your comprehensive Word document is now saved to your PC!

🎉 ANALYSIS COMPLETE - ENJOY YOUR REPORT!

📊 REPORT CONTENTS:
   • Executive Summary with complete KPI table
   • Trade Balance Analysis with annual data table
   • Growth Dynamics with CAGR charts
   • Coverage Analysis with trend visualization
   • Volatility Assessment
   • Re-export Dynamics
   • BCG Portfolio Matrix with product classification
   • HHI Concentration Analysis
   • Top Products Tables (Stars, Top 10 exports/imports)
   • Strategic Recommendations (15+ actionable strategies)
   • 3-Phase Implementation Roadmap (20+ specific actions)
   • Professional charts embedded throughout

